# Implementing Vision Transformer from Scratch

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import layers
from keras.datasets import cifar100
import tensorflow_addons as tfa

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')

# Hyperparameters

In [ ]:
BATCH_SIZE = 256
EPOCH = 2

# Data

In [9]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)
